**INSTALL LIBRARIES**

In [1]:
import pandas as pd
import geopandas as gpd

import numpy as np

import folium
from folium.plugins import TimestampedGeoJson

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors



**READ DATA**

In [17]:
#pandas
inno_geocoded = pd.read_csv("inno_geocoded.csv")

inno_geocoded["lat"].fillna(inno_geocoded["latitude"])
inno_geocoded["lng"].fillna(inno_geocoded["longitude"])
#geopandas
gdf = gpd.GeoDataFrame(inno_geocoded, geometry=gpd.points_from_xy(inno_geocoded.lng, inno_geocoded.lat))

c:\Users\eric9\anaconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**MOVE LONGITUDE LATITUDE TO LNG LAT COLUMN**

**FILTER FOR INNO**

In [18]:
inno2 = gdf["inno_nr"] == "inno_31"
inno2 = gdf[inno2]
inno2 = inno2[inno2["lat"].notna()] # only needed, until step above is coded

#change datatype of createad at to datetime
inno2["created_at_td"] = pd.to_datetime(inno2["created_at"])

**ADD COLOR COLUMN TO DATAFRAME**

In [20]:
inno2["created_at_int"] = inno2["created_at_td"].apply(lambda x:x.toordinal())
min = inno2["created_at_int"].min()
max = inno2["created_at_int"].max()
norm = matplotlib.colors.Normalize(vmin= min,vmax= max, clip=True)
mapper = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis) #viridis can be changed for other color palette
inno2["color"] = inno2["created_at_int"].apply(lambda x: mcolors.to_hex(mapper.to_rgba(x)))

**PREPARE LINES FOR TIMESTAMPED GEOJSON ANIMATION**

Coordinates

In [21]:
#nested list with coordinates defining start and end of each line
x_coords = inno2["geometry"].x.to_list()
y_coords = inno2["geometry"].y.to_list()

coordinate_list_1 = [[x, y] for x, y in zip(x_coords, y_coords)]
coordinate_list_2 = [[x, y] for x, y in zip(x_coords, y_coords)]

#remove first element(coordinate) of second list
del coordinate_list_2[0]

coordinate_list_1

coords = [list(i) for i in zip(coordinate_list_1, coordinate_list_2)]


Time

In [22]:
time = []
for i in inno2["created_at"]:
    sublist = [i,i]
    time.append(sublist)


Color

In [23]:
color = inno2["color"].to_list()

In [24]:
#list with dicts containing coordinates, date/time and color information for each line
lines = []
for i,j,k in zip(coords,time,color):
    
    new_item = {}
    new_item['coordinates'] = i
    new_item["dates"] = j
    new_item["color"] = k

    lines.append(new_item)

**VISUALISATION**

In [39]:
m = folium.Map([47.32, 8.50], zoom_start=2, tiles = "CartoDB dark_matter", control_scale = True)


features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {"color": line["color"],"stroke-width": 3},
            "icon": "circle",
            "iconstyle": {"fillColor": line["color"], "fillOpacity": 1, "stroke": "true", "radius": 4}
        },
    }
    for line in lines
]

TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="P1D", # 'P1Y' 1/year, P1M' 1/month, 'P1D' 1/day, 'PT1H' 1/hour, and 'PT1M' 1/minute
    add_last_point=True,
).add_to(m)

m